In [1]:
import numpy as np
import pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

### Model 1 - Linear Regression ###

In [2]:
#load train test and validation data from tensor_collection
cm_x_train = torch.load('tensor_collection\\cm_x_train.pt')
cm_y_train = torch.load('tensor_collection\\cm_y_train.pt')
cm_x_test = torch.load('tensor_collection\\cm_x_test.pt')
cm_y_test = torch.load('tensor_collection\\cm_y_test.pt')
cm_x_val = torch.load('tensor_collection\\cm_x_val.pt')
cm_y_val = torch.load('tensor_collection\\cm_y_val.pt')



In [3]:
#Loss function
def mean_squared_error(x : np.ndarray, y : np.ndarray, theta : np.ndarray) -> np.ndarray:
    yhat = x @ theta 
    error = yhat - y 
    loss = (1 / len(y)) * np.sum(error ** 2) 
    return loss

#Gradient descent
def calculate_gradient_and_update(x: np.ndarray, y: np.ndarray, theta: np.ndarray, alpha: float) -> tuple([float, np.ndarray]):
    gradient = (1 / len(y)) * x.T @ ((x @ theta) - y) 
    theta_new = theta - (alpha * gradient) 
    loss = mean_squared_error(x, y, theta_new)
    return loss, theta_new


In [4]:
num_features = cm_x_train.shape[1]

train_loss_history = []
val_loss_history = []
num_epochs = 10000
alpha = 0.1

# Initialize theta to random values between -2 and 2
theta = np.random.uniform(-2, 2, (num_features))

# Training Loop
for t in range(num_epochs):
    train_loss, theta = calculate_gradient_and_update(cm_x_train, cm_y_train, theta, alpha)
    train_loss_history.append(train_loss)

    # Validation step
    val_loss, _ = calculate_gradient_and_update(cm_x_val, cm_y_val, theta, 0)  # alpha=0 to prevent updates
    val_loss_history.append(val_loss)


# Plot loss history
print('Final training loss:', train_loss)
print('Final validation loss:', val_loss)


Final training loss: 66.8928832558388
Final validation loss: 63.11712690253444


In [5]:
#save theta and loss history for analysis notebook
pickle.dump(theta, open('tensor_collection\\regression_theta.pkl', 'wb'))
pickle.dump(train_loss, open('tensor_collection\\regression_train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('tensor_collection\\regression_val_loss.pkl', 'wb'))

### Model 2 - Training CNN ###

In [6]:
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", cuda_available)

Is CUDA available?  True


In [7]:
#load torch tensors
#load from csv
directory = 'tensor_collection\\'

#load train and val csv files
cnn_x_train = pd.read_csv(directory + 'cnn_x_train.csv')
cnn_x_val = pd.read_csv(directory + 'cnn_x_val.csv')


cnn_y_train_tensor = torch.load(directory + 'cnn_y_train.pt')
cnn_y_val_tensor = torch.load(directory + 'cnn_y_val.pt')

#print types
print(type(cnn_x_train))
print(type(cnn_y_train_tensor))



<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [8]:
if (torch.cuda.is_available()):
    device = torch.device("cuda:0")
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('GPU not available, using CPU')
    

Using GPU: NVIDIA GeForce GTX 1660 Ti


In [9]:
from dataloader import CustomDataloader
from networks import CNNRegression
import tqdm

In [10]:


train_dataloader = CustomDataloader(dataframe=cnn_x_train, age=cnn_y_train_tensor, batch_size=50, randomize=True)
val_dataloader = CustomDataloader(dataframe=cnn_x_val, age=cnn_y_val_tensor, batch_size=50, randomize=False)

model = CNNRegression().to(device)

# instantiate your optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# log your losses
train_losses = []
val_losses = []

# define how many epochs to train on
epochs = 50

def save_checkpoint(model, optimizer, epoch, filename):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, filename)

# define your loss function for multiclass classification task
# BCE does binary cross entropy automatically for each class
loss_fn = torch.nn.MSELoss()
for _ in tqdm.tqdm(range(epochs)):
    model.train()
    train_losses_epoch = []
    for _ in range(train_dataloader.num_batches_per_epoch):
        optimizer.zero_grad()
        train_batch = train_dataloader.fetch_batch()
        x_batch = train_batch['img_batch'].to(device)
        y_batch = train_batch['age_batch'].to(device)
        yhat = model(x_batch)  # Use only image batch
        yhat = torch.squeeze(yhat)
        train_loss = torch.mean(loss_fn(yhat, y_batch))  # Adjusted to label_batch

        # Training data backward pass
        train_loss.backward()
        optimizer.step()
        train_losses_epoch.append(train_loss.detach().cpu().numpy())
    
    train_losses.append(np.mean(train_losses_epoch))
    
    model.eval()
    val_losses_epoch = []
    with torch.no_grad():
        for _ in range(val_dataloader.num_batches_per_epoch):
            # Validation data forward pass only
            val_batch = val_dataloader.fetch_batch()
            x_batch = val_batch['img_batch'].to(device)
            y_batch = val_batch['age_batch'].to(device)
            
            yhat = model(x_batch)  # Use only image batch
            yhat = torch.squeeze(yhat)
            val_loss = torch.mean(loss_fn(yhat, y_batch), axis=0)  # Adjusted to label_batch
            val_losses_epoch.append(val_loss.detach().cpu().numpy())

    val_losses.append(np.mean(val_losses_epoch))
    
    # Save checkpoint after each epoch
    checkpoint_filename = f'model_checkpoints/cnn_checkpoint_epoch_{epoch}.pth'
    save_checkpoint(model, optimizer, epoch, checkpoint_filename)

100%|██████████| 50/50 [08:22<00:00, 10.05s/it]


In [11]:
#store train and validation losses
torch.save(train_losses, 'tensor_collection\\cnn_train_losses.pt')
torch.save(val_losses, 'tensor_collection\\cnn_val_losses.pt')
#store epochs for plotting
torch.save(epochs, 'tensor_collection\\cnn_epochs.pt')



In [14]:
from networks import MultiModalNetwork  # Use the multimodal network class

#get number of numerical features
num_numerical_features = cnn_x_train.shape[1] - 1
print(cnn_x_train.head())

train_dataloader = CustomDataloader(dataframe=cnn_x_train, age=cnn_y_train_tensor, batch_size=50, randomize=True)
val_dataloader = CustomDataloader(dataframe=cnn_x_val, age=cnn_y_val_tensor, batch_size=50, randomize=False)

model = MultiModalNetwork(num_numerical_features).to(device)  # Adjust for the number of numerical features

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train_losses = []
val_losses = []
epochs = 50
loss_fn = torch.nn.MSELoss()

def save_checkpoint(model, optimizer, epoch, filename):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, filename)

for epoch in tqdm.tqdm(range(epochs)):
    model.train()  # Set the model to training mode
    train_losses_epoch = []
    for _ in range(train_dataloader.num_batches_per_epoch):
        optimizer.zero_grad()
        train_batch = train_dataloader.fetch_batch()
        x_batch = train_batch['img_batch'].to(device)
        num_features = train_batch['feat_batch'].to(device)  # Fetch numerical features
        y_batch = train_batch['age_batch'].to(device)
        
        yhat = model(x_batch, num_features)  # Pass both image and numerical data
        yhat = torch.squeeze(yhat)
        train_loss = loss_fn(yhat, y_batch)
        
        train_loss.backward()
        optimizer.step()
        train_losses_epoch.append(train_loss.item())
    
    train_losses.append(np.mean(train_losses_epoch))
    
    model.eval()  # Set the model to evaluation mode
    val_losses_epoch = []
    with torch.no_grad():
        for _ in range(val_dataloader.num_batches_per_epoch):
            val_batch = val_dataloader.fetch_batch()
            x_batch = val_batch['img_batch'].to(device)
            num_features = val_batch['feat_batch'].to(device)  # Fetch numerical features
            y_batch = val_batch['age_batch'].to(device)
            
            yhat = model(x_batch, num_features)  # Pass both image and numerical data
            yhat = torch.squeeze(yhat)
            val_loss = loss_fn(yhat, y_batch)
            val_losses_epoch.append(val_loss.detach().cpu().numpy())

    val_losses.append(np.mean(val_losses_epoch))
    
    # Save checkpoint after each epoch
    checkpoint_filename = f'model_checkpoints/mmn_checkpoint_epoch_{epoch}.pth'
    save_checkpoint(model, optimizer, epoch, checkpoint_filename)

   num_haircuts_life  has_tiktok  remembers_disco  uses_skincare  \
0                229           1                0              0   
1                237           0                0              0   
2                346           0                0              1   
3                390           1                0              0   
4                 84           0                0              0   

   max_annual_earnings  race_asian  race_black  race_indian  race_other  \
0        186190.884193           0           0            1           0   
1        292089.985703           0           1            0           0   
2        330816.556160           0           0            0           0   
3         76057.894571           0           0            0           0   
4          2609.930728           0           0            0           0   

   race_white  gender_female  gender_male  bias  \
0           0              0            1     1   
1           0              1          

 34%|███▍      | 17/50 [02:47<05:25,  9.85s/it]


KeyboardInterrupt: 

In [13]:
#store train and validation losses
torch.save(train_losses, 'tensor_collection\\mmn_train_losses.pt')
torch.save(val_losses, 'tensor_collection\\mmn_val_losses.pt')
#store epochs for plotting
torch.save(epochs, 'tensor_collection\\mmn_epochs.pt')

